In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain.text_splitter'

In [4]:
pip install langchain-text-splitters


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool
import vertexai

import os

os.environ["GRPC_VERBOSITY"] = ""
os.environ["GRPC_TRACE"] = ""

# Create a RAG Corpus, Import Files, and Generate a response

# TODO(developer): Update and un-comment below lines
PROJECT_ID = "bing-tan-sndbx-c"
display_name = "personal-documents"
paths = ["gs://rag-corpus-bing/documents"]  # Supports Google Cloud Storage and Google Drive Links

# Initialize Vertex AI API once per session
vertexai.init(project=PROJECT_ID, location="europe-west4")

# Create RagCorpus
# Configure embedding model, for example "text-embedding-005".
embedding_model_config = rag.RagEmbeddingModelConfig(
    vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
        publisher_model="publishers/google/models/text-embedding-005"
    )
)

rag_corpus = rag.get_corpus(name="projects/bing-tan-sndbx-c/locations/europe-west4/ragCorpora/6917529027641081856")


# Import Files to the RagCorpus
rag.import_files(
    rag_corpus.name,
    paths,
    # Optional
    transformation_config=rag.TransformationConfig(
        chunking_config=rag.ChunkingConfig(
            chunk_size=512,
            chunk_overlap=100,
        ),
    ),
    max_embedding_requests_per_min=1000,  # Optional
)

# Direct context retrieval
rag_retrieval_config = rag.RagRetrievalConfig(
    top_k=3,  # Optional
    filter=rag.Filter(vector_distance_threshold=0.5),  # Optional
)
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="Tell me who is databricks academy ml ops certified?",
    rag_retrieval_config=rag_retrieval_config,
)
print(response)


# Enhance generation
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            rag_retrieval_config=rag_retrieval_config,
        ),
    )
)

# Create a Gemini model instance
rag_model = GenerativeModel(
    model_name="gemini-2.0-flash-001", tools=[rag_retrieval_tool]
)

# Generate response
response = rag_model.generate_content("Tell me who is databricks academy ml ops certified?")
print(response.text)

E0000 00:00:1760531627.785267 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760531627.786294 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760531628.356233 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760531628.357249 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760531635.847730 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760531635.849580 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


contexts {
  contexts {
    source_uri: "gs://rag-corpus-bing/documents/XD Team Update - September 2025.pdf"
    text: "A new avenue \r\nto collaborate is the DPP program. Colleagues that get onboarded through the program can \r\nbe selected by Databricks to help their customers. \r\nThere are various specializations and levels, from Data Engineering to AI/ML, from \r\nArchitects to Solution Consultants. Currently, Databricks is asking us to propose Sr. GenAI \r\nEngineers, and Caio have started the interview process. \r\nOf the colleagues already onboarded, Victor K has started at ASML through the DPP \r\nprogram, while Cor has started at Allianz. \r\nWe plan to continue to onboard new colleagues and keep on growing the partnership \r\nthrough regular meetings with Databricks Sales and Databricks partner managers in \r\nEurope, while also increasing our pool of certified consultants & Databricks champions \r\n(which counts towards our partner status: we’re currently Elite, with 83 cer

/Users/bing/.pyenv/versions/3.10.18/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1760531642.417863 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760531642.420945 2579195 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Bing Tan is certified in Machine Learning Operations by Databricks Academy. Xebia has 83 certified colleagues.
